In [135]:
import pandas as pd
from selenium import webdriver
import chromedriver_binary
from datetime import date
import warnings
warnings.filterwarnings('ignore')
import time
from bs4 import BeautifulSoup

In [56]:
df = pd.read_excel('link_by_areas.xlsx')

In [112]:
links = df.iloc[:, 1]

In [142]:
soup_results=[]
for link in links:
    driver = webdriver.Chrome()
    driver.get(link)
    soup = BeautifulSoup(driver.page_source)
    soup_results.append(soup.find_all('article', {'class':'jobTuple'}))
    driver.close()

In [143]:
soup_results

[[<article class="jobTuple" data-job-id="010423005090"><div class="jobTupleHeader"><div class="info fleft"><a class="title ellipsis" href="https://www.naukri.com/job-listings-taxation-and-accounts-assistant-b-james-co-mumbai-2-to-4-years-010423005090" target="_blank" title="Taxation And Accounts Assistant">Taxation And Accounts Assistant</a><div class="companyInfo subheading"><a class="subTitle ellipsis fleft" href="https://www.naukri.com/b-james-co-jobs-careers-2672648" target="_blank" title="B James &amp; Co">B James &amp; Co</a></div></div><ul class=""><li class="fleft br2 placeHolderLi experience"><i class="fleft placeholder-icons naukicon naukicon-srp-experience"></i><span class="ellipsis fleft expwdth" title="2-4 Yrs ">2-4 Yrs</span><span class="separator"></span></li><li class="fleft br2 placeHolderLi salary"><i class="fleft placeholder-icons naukicon naukicon-srp-rupee"></i><span class="ellipsis fleft" title="Not disclosed ">Not disclosed</span><span class="separator"></span></

In [146]:
flattened_soup_results = [element for inner_list in soup_results for element in inner_list]

In [147]:
len(flattened_soup_results)

884

In [148]:
from codecs import ignore_errors


df_jobs = pd.DataFrame()
for soup in flattened_soup_results:
    obj = soup
    job_title = obj.find('a','title ellipsis').text
    experience = obj.find('span','ellipsis fleft expwdth')
    if experience is not None:
        experience = experience.text
    company =  obj.find('a', 'subTitle ellipsis fleft').text
    scraped_date = date.today().strftime("%d/%m/%Y")
    salary = obj.find('span', 'ellipsis fleft').text
    location = obj.find('span', "ellipsis fleft locWdth")
    if location is not None:
        location = location.text
    tags = obj.find_all('li',{'class':'fleft dot'})
    tag_list = []
    for tag in tags:
        tag_list.append(tag.text)
    posted_date = obj.find('span', 'fleft postedDate').text
    df_jobs=df_jobs.append({'Job Title':job_title, 'Experience':experience, 'Company':company ,'Scraped data':scraped_date, 'Salary':salary,
                            'Location':location, 'Tags':tag_list, 'Posted date':posted_date}, ignore_index=True)
df_jobs.head()

,Job Title,Experience,Company,Scraped data,Salary,Location,Tags,Posted date
0,Taxation And Accounts Assistant,2-4 Yrs,B James & Co,07/04/2023,Not disclosed,Mumbai,"[accounting, taxation, finalisation, tax retur...",5 Days Ago
1,Accounts Executive - Payables,0-3 Yrs,Hilton Hotels,07/04/2023,Not disclosed,Panaji/Panjim,"[supervision, accounting, balance sheet, payab...",1 Day Ago
2,Senior Accounts Executive,0-2 Yrs,Rounak Jain Associates,07/04/2023,2-3 Lacs PA,Thane,"[Gst Filing, accounting, filing, gst reconcili...",1 Day Ago
3,Finance And Accounts Executive,0-5 Yrs,ALLENGERS GLOBAL HEALTHCARE PRIVATE LIMITED,07/04/2023,Not disclosed,"Mohali, Chandigarh, Kharar, Naya Gaon, Panchku...","[Financial Accounting, reporting, accounts fin...",2 Days Ago
4,Finance And Accounts Executive,0-5 Yrs,ALLENGERS GLOBAL HEALTHCARE PRIVATE LIMITED,07/04/2023,Not disclosed,"Mohali, Chandigarh, Kharar, Naya Gaon, Panchku...","[Financial Accounting, reporting, accounts fin...",2 Days Ago


In [ ]:
df_jobs.to_csv('jobs.csv', index=False)